In [9]:
# No 1
import sqlite3
import pandas as pd
from scipy.stats import pearsonr

# Connect to the database
conn = sqlite3.connect(r'C:\Users\Oluwafunmilayo Basil\Documents\EBUNOLUWA\AdventureWork_Database.db')

# Query to extract product prices and sales quantities
query = '''
SELECT ProductPrice, SUM(OrderQuantity) AS TotalQuantity
FROM Sales_Products_2015_2017
GROUP BY ProductKey
'''

# Load data into a pandas DataFrame
df = pd.read_sql(query, conn)

print(df)

     ProductPrice  TotalQuantity
0         34.9900           2099
1         33.6442           1940
2         33.6442           1995
3          8.6442           4151
4         48.0673            392
..            ...            ...
125      539.9900             55
126      539.9900             41
127      539.9900            356
128      539.9900            355
129      539.9900            383

[130 rows x 2 columns]


In [11]:
# Perform Pearson correlation between ProductPrice and TotalQuantity
corr, p_value = pearsonr(df['ProductPrice'], df['TotalQuantity'])

# Output the results
print(f"Pearson Correlation Coefficient: {corr}")
print(f"P-value: {p_value}")

# Interpret the results
if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant negative correlation.")
else:
    print("Fail to reject the null hypothesis: There is no significant correlation.")


Pearson Correlation Coefficient: -0.3630292079645787
P-value: 2.1865231911233068e-05
Reject the null hypothesis: There is a significant negative correlation.


In [5]:

import sqlite3
import pandas as pd
from scipy import stats

# Connect to the database
conn = sqlite3.connect(r'C:\Users\Oluwafunmilayo Basil\Documents\EBUNOLUWA\AdventureWork_Database.db')

# Query to extract revenue and territory information
query = '''
SELECT TerritoryKey, SUM(Revenue) AS TotalRevenue
FROM Sales_Products_2015_2017
GROUP BY OrderID, TerritoryKey
'''

# Load data into a pandas DataFrame
df = pd.read_sql(query, conn)

# Perform ANOVA test to compare sales across different territories
territories = df.groupby('TerritoryKey')['TotalRevenue'].apply(list)
anova_result = stats.f_oneway(*territories)

# Output the results
print(f"F-statistic: {anova_result.statistic}")
print(f"P-value: {anova_result.pvalue}")

# Interpret the results
if anova_result.pvalue < 0.05:
    print("Reject the null hypothesis: Sales performance varies significantly across territories.")
else:
    print("Fail to reject the null hypothesis: Sales performance does not vary significantly across territories.")


F-statistic: 101.89472870164182
P-value: 4.0454702505275955e-188
Reject the null hypothesis: Sales performance varies significantly across territories.


In [7]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Connect to the database
conn = sqlite3.connect(r'C:\Users\Oluwafunmilayo Basil\Documents\EBUNOLUWA\AdventureWork_Database.db')

# Query to extract product categories and order dates
query = '''
SELECT ProductCategoryKey, Revenue, strftime('%m', OrderDate) AS Month
FROM Sales_Products_2015_2017
'''

# Load data into a pandas DataFrame
df = pd.read_sql(query, conn)

# Create a 'Season' column based on the 'Month' (Spring: 03-05, Summer: 06-08, Fall: 09-11, Winter: 12-02)
def get_season(month):
    month = int(month)
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['Month'].apply(get_season)

# Create a contingency table for product categories and seasons
contingency_table = pd.crosstab(df['ProductCategoryKey'], df['Season'])

# Perform Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Output the results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")

# Interpret the results
if p < 0.05:
    print("Reject the null hypothesis: There is a significant association between product category and seasonal sales.")
else:
    print("Fail to reject the null hypothesis: No significant association between product category and seasonal sales.")


Chi-Square Statistic: 250.0797607847409
P-value: 3.943512611645011e-51
Reject the null hypothesis: There is a significant association between product category and seasonal sales.


In [13]:
import sqlite3
import pandas as pd
from scipy.stats import shapiro, f_oneway, kruskal

# Connect to the database
conn = sqlite3.connect(r'C:\Users\Oluwafunmilayo Basil\Documents\EBUNOLUWA\AdventureWork_Database.db')

# Query to extract revenue by territory
query = '''
SELECT TerritoryKey, SUM(Revenue) AS TotalRevenue
FROM Sales_Products_2015_2017
GROUP BY OrderID, TerritoryKey
'''

# Load data into a pandas DataFrame
df = pd.read_sql(query, conn)

# Check normality for TotalRevenue using Shapiro-Wilk test for each territory
territories = df['TerritoryKey'].unique()
normality_results = {}

for territory in territories:
    territory_data = df[df['TerritoryKey'] == territory]['TotalRevenue']
    normality_results[territory] = shapiro(territory_data).pvalue

# If any of the territories fails the normality test, use Kruskal-Wallis
use_kruskal = any(p < 0.05 for p in normality_results.values())

if use_kruskal:
    # Perform Kruskal-Wallis test
    grouped_territories = [df[df['TerritoryKey'] == t]['TotalRevenue'] for t in territories]
    stat, p_value = kruskal(*grouped_territories)
    test_used = 'Kruskal-Wallis Test'
else:
    # Perform ANOVA
    grouped_territories = [df[df['TerritoryKey'] == t]['TotalRevenue'] for t in territories]
    stat, p_value = f_oneway(*grouped_territories)
    test_used = 'ANOVA'

# Output the results
print(f"{test_used}: Statistic = {stat}, p-value = {p_value}")


Kruskal-Wallis Test: Statistic = 959.4093701992699, p-value = 9.723901414974708e-201


C:\Users\Oluwafunmilayo Basil\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6060.
  res = hypotest_fun_out(*samples, **kwds)
